In [1]:
import os
import time
import sys
from multiprocessing.pool import ThreadPool
from contextlib import closing

# pip install config-file requests sine.utils
import requests
from config_file import ConfigFile, ParsingError
from sine.utils import *

In [8]:
def download_pic0(url, path):
    resp = requests.get(url)
    with open(path, 'wb') as code:
        code.write(resp.content)
def download_pic1(url, path, bar):
    resp = requests.get(url)
    with open(path, 'wb') as code:
        bar.load_in(len(resp.content))
        code.write(resp.content)
        bar.update_out(len(resp.content))
def download_pic(url, path, bar):
    with closing(requests.get(url, stream=True)) as resp:
        size = int(resp.headers['Content-Length'])
        bar.load_in(size)
        with open(path, 'wb') as code:
            for chunk in resp.iter_content(chunk_size=1024*config.getAlone('buffer_KB', 1000)):
                code.write(chunk)
                bar.update_out(len(chunk), len(chunk)/size)
                bar.set_postfix_str(byte2readable(bar.loaded_avg * bar.n / (time.time() - bar.start_t)) + '/s')
def download_pic_batch(url, path, count):
    mkdir(path)
    print('开始从 %s 批量下载 %d 张图片到 %s'% (url, count, os.path.abspath(path)))
    urls=[]
    paths=[]
    for i in range(0, count):
        urls.append(url+'?&'+str(i))
        filename = 'MirlKoi-' + time.strftime('%Y%m%d%H%M%S-') + str(i) + '.jpg'
        paths.append(os.path.join(path, filename))
    with ThreadPool(config.getAlone('thread_pool', 20)) as executor, PredictableProcessBar(total=len(paths), unit='张', smoothing=0) as bar:
        for i in range(0, len(paths)):
            executor.apply_async(download_pic, args=(urls[i], paths[i], bar), callback=None)
        executor.close()
        executor.join()
        bar.display()
        time.sleep(1)
        bar.display()

In [9]:
if __name__ == '__main__':
    config_filename = "MirlKoi-download_config.json"
else:
    config_filename = __name__ + "_config.json"
try:
    config = ConfigFileAlone(config_filename)
    save_path = config.getAlone('save_path', 'MirlKoi')
    api_path = config.getAlone('api_path', 'https://iw233.cn/API/Random.php')
    download_num = config.getAlone('download_num', 20)
    download_pic_batch(api_path, save_path, download_num)
    config.saveAlone()
except Exception:
    print("无法读取配置文件 " + config_filename)

开始从 https://iw233.cn/API/Random.php 批量下载 20 张图片到 D:\workspace\ipynb-workspace\MirlKoi


100%|█████████████████████████████████████████████████████████████████████| 20.00/20 [00:35<00:00,  1.75s/张, 2.30MB/s]
